In [ ]:
# Part3_Loops_and_Agents.ipynb

from typing import TypedDict, Annotated, List, Dict
from langgraph.graph import StateGraph
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
import traceback

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=None
)

# === State Definition ===
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    code: str
    result: str
    iterations: int
    improvements: List[str]
    success: bool

# === Core Agents ===
def planning_agent(state: AgentState, llm: ChatOpenAI = llm):
    """Planning agent that designs solutions."""
    system_prompt = """
    Based on the user request and context:
    1. Analyze the requirements
    2. Break down into steps
    3. Consider potential issues
    4. Create actionable plan
    """
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {
        "messages": [response],
        "iterations": state.get("iterations", 0) + 1
    }

def execution_agent(state: AgentState, llm: ChatOpenAI = llm):
    """Execution agent that generates code."""
    system_prompt = """
    Transform the plan into executable code:
    1. Follow the plan steps
    2. Generate appropriate code
    3. Include error handling
    4. Add documentation
    """
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {
        "code": response.content,
        "messages": [response]
    }

def review_agent(state: AgentState, llm: ChatOpenAI = llm):
    """Review agent that evaluates output."""
    system_prompt = """
    Review the output and decide:
    1. Is it correct? (#CORRECT/#INCORRECT)
    2. Are there improvements needed?
    3. What specific changes would help?
    """
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {
        "messages": [response],
        "improvements": state.get("improvements", []) + [response.content]
    }

# === Routing Logic ===
def feedback_router(state: AgentState):
    """Route based on review feedback."""
    last_message = state["messages"][-1].content
    if "#INCORRECT" in last_message:
        return "revise"
    return "complete"

def error_router(state: AgentState):
    """Route based on error type."""
    if not state.get("success", False):
        error_type = analyze_error(state.get("error", ""))
        return {
            "syntax": "fix_syntax",
            "logic": "fix_logic",
            "data": "fix_data"
        }.get(error_type, "plan")
    return "continue"

# === Error Handling ===
def detect_errors(state: AgentState):
    """Detect errors in execution."""
    try:
        # Attempt to execute or validate code
        if state.get("code"):
            # Add your code execution logic here
            result = "Success"  # Placeholder
            return {"success": True, "result": result}
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "messages": [HumanMessage(content=f"Error detected: {str(e)}")]
        }
    return {"success": True}

def analyze_error(error_message: str) -> str:
    """Analyze error type."""
    if "syntax" in error_message.lower():
        return "syntax"
    elif "logic" in error_message.lower():
        return "logic"
    return "unknown"

# === Graph Creation ===
def create_agent_system():
    """Create complete agent system with feedback loops."""
    graph = StateGraph(AgentState)
    
    # Add nodes
    graph.add_node("plan", planning_agent)
    graph.add_node("execute", execution_agent)
    graph.add_node("review", review_agent)
    graph.add_node("error_check", detect_errors)
    
    # Add edges
    graph.add_edge("plan", "execute")
    graph.add_edge("execute", "error_check")
    
    # Add conditional edges
    graph.add_conditional_edges(
        "error_check",
        error_router,
        {
            "fix_syntax": "execute",
            "fix_logic": "plan",
            "fix_data": "plan",
            "continue": "review"
        }
    )
    
    graph.add_conditional_edges(
        "review",
        feedback_router,
        {
            "revise": "plan",
            "complete": "end"
        }
    )
    
    return graph.compile()

# === Example Usage ===
def run_example():
    """Run example with agent system."""
    # Initialize system
    system = create_agent_system()
    
    # Initial state
    initial_state = AgentState(
        messages=[HumanMessage(content="Create a function to calculate the average of a list of numbers")],
        code="",
        result="",
        iterations=0,
        improvements=[],
        success=False
    )
    
    # Run system
    try:
        result = system.invoke(initial_state)
        print("Final Result:", result)
    except Exception as e:
        print(f"Error: {str(e)}")
        print(traceback.format_exc())

# === Exercises ===

def exercise_1():
    """
    Exercise 1: Implement a simple feedback loop
    - Create generator agent
    - Create reviewer agent
    - Implement loop until quality threshold met
    """
    # Your code here
    pass

def exercise_2():
    """
    Exercise 2: Implement error recovery
    - Create error detection
    - Implement recovery routes
    - Track recovery attempts
    """
    # Your code here
    pass

def exercise_3():
    """
    Exercise 3: Build complete self-improving agent
    - Implement all components
    - Add feedback loops
    - Track improvements
    """
    # Your code here
    pass

if __name__ == "__main__":
    # Run example
    print("Running agent system example...")
    run_example()